# Central Sulcus SNR and Information Capacity Mapping
Draws a simple central sulcus and simulates a depth electrode on both sides.


## Libraries and assigned values


In [ ]:
### Import libraries ###
"""Workflow Overview:
1. Load precomputed lead fields for a DiSc array.
2. Construct an analytic 2D profile approximating a deep central sulcus and derive outward normals.
3. Place two depth devices (simulated electrodes) on opposite sulcal banks; rotate local dipole coordinate frames.
4. For each sulcal surface sample and depth position, project dipole vectors through lead fields to get electrode voltages.
5. Derive DiSc and aggregated SEEG SNR maps; optionally apply montage (difference enhancement) logic.
6. Convert SNR to Shannon–Hartley information capacity and visualize 3D surfaces and 2D heatmaps.
7. Report summary metrics (max, total capacity, spatial ratios).

Key Objects:
- fields: 5D array [X,Z,Y,vector_component,electrode] of lead field values.
- dippos: Integer voxel indices for dipole positions mapped into leadfield space.
- dipvec: 3D dipole moment vectors (scaled by magnitude) per surface sample.
- v / vseeg: Voltage tensors for DiSc sensors and aggregated SEEG rings.

Adjust 'folder' to point at the dataset root. Ensure the .npz file path is valid before executing downstream cells.
"""
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib import cm
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from os import path
import sys
sys.path.insert(0, path.join('.'))
from modules.leadfield_importer import FieldImporter

# Select dataset directory
# folder = r"...\SEPIO_dataset"  # Example placeholder; replace with actual dataset path
folder = r"B:\NEI\Test"  # Active dataset directory (modify if relocating project)
fields_file = path.join(folder,'leadfields', 'DISC_30mm_p2-5Sm_MacChr.npz')

# Output directory for generated figures
output = path.join(folder,'outputs')


# Get lead fields
field_importer = FieldImporter()
field = field_importer.load(fields_file)
num_electrodes = np.shape(field_importer.fields)[4]
fields = field_importer.fields  # Shape: [Nx, Nz, Ny, 3(vector basis), Nelectrodes]
scale = 0.5  # mm per voxel

# Dipole model parameters
magnitude = 0.5e-9  # nAm; dipole moment magnitude
# Noise values (rms, microvolts); adapt for empirical device characterization
disc_noise = 4.1  # DiSc typical rms noise (uV)
seeg_noise = 2.3  # SEEG typical rms noise (uV)


## Draw section and determine values

In [ ]:
### Calculate curve and normal vectors ###
"""Sulcus Geometry Construction:
We approximate a central sulcus cross-section using y(x) = -7.5*cos(x^2/3) - 7.5, producing a deep concavity.
Sampling:
- x in [-3, 3] with xnum points.
Normals:
- For each x, derive an unnormalized perpendicular vector [-dy, dx].
- Normalize to unit length for consistent dipole orientation scaling.
These normals represent cortical column orientations emerging from the sulcal wall.
"""
# A curve meant to emulate a deep sulcus (inner cortical surface)
xstart = -3
xstop = 3
xnum = 100
dx = (xstop-xstart)/xnum
x = np.linspace(xstart,xstop,xnum)
y = -7.5*np.cos((x**2)/3) - 7.5
vec = np.zeros((y.shape[0],2)) # [point,[-dy,dx]]
for i,xi in enumerate(x): # collect normal, unit vectors from y(x)
    vec[i] = np.array([(-y[i]+y[i-1]),dx])
    vec[i] *= 1/np.sqrt(vec[i,0]**2 + vec[i,1]**2)
vec[0] = vec[1]  # First point finite difference edge correction

# Quick diagnostic plot
plt.plot(x,y)
plt.xlim([-7,7])
test = np.meshgrid(x,y)
plt.quiver(x,y,vec[:,0],vec[:,1])
plt.title("Sulcus Profile with Normal Vectors")
plt.show()

In [ ]:
### Place devices and transform vector space for each
"""Device Placement & Coordinate Transform:
Two depth devices are defined by line segments (start->end) in millimeter space (devpos). For each device:
1. Translate sulcus sample coordinates (x,y) into a local frame centered at the device midpoint.
2. Compute device axis angle (devangle) from its segment; rotate normals so dipole vectors align with device insertion trajectory.
3. Map rotated positions into discrete leadfield voxel indices (dippos) using 'scale' and centering offset.
4. Rotate normal vectors for each device independently to produce dipvec (later expanded to 3D). Dipole directions remain unit length before scaling by magnitude.
Rationale:
- Independent rotation preserves anatomical orientation differences between devices.
- Using integer indices ensures safe direct array indexing into 'fields'.
Potential extension: Vectorized rotation and normalization instead of Python loops for speed.
Outputs:
- dippos: int voxel indices [device, (x,y), sample]
- dipvec: rotated 2D dipole direction vectors [device, (nx, ny), sample]
"""
# Modify for device positions; [x,y] for the start and end of each device
devpos = np.array([
    [[-4.5,0],[-3,-14.5]],  # Device 0 near sulcal center
    [[4,0],[2.5,-14.5]]     # Device 1 opposite bank
])

# Translate & rotate vector locations to each device relative (still in mm)
dippos = np.zeros((2,2,y.shape[0])) # [device,x-y,point(x index)]
dippos[0] = np.array([x-np.mean(devpos[0],axis=0)[0],y-np.mean(devpos[0],axis=0)[1]])
dippos[1] = np.array([x-np.mean(devpos[1],axis=0)[0],y-np.mean(devpos[1],axis=0)[1]])
devangle = -np.pi/2 - np.array([np.arctan2((devpos[0,1,1]-devpos[0,0,1]),devpos[0,1,0]-devpos[0,0,0]),
                     np.arctan2((devpos[1,1,1]-devpos[1,0,1]),devpos[1,1,0]-devpos[1,0,0])])
R = np.array([[np.cos(devangle),-np.sin(devangle)],
              [np.sin(devangle),np.cos(devangle)]])
dippos[0] = np.dot(R[:,:,0],dippos[0])
dippos[1] = np.dot(R[:,:,1],dippos[1])
# Transfer position into leadfield space
dippos *= 1/scale
dippos += fields.shape[0]//2  # assume cubic grid; shift to center
# Integer casting for array indexing
dippos = dippos.astype('int')

# Rotate vectors for each device; [device,x-y,point(x index)]
dipvec = np.zeros((2,vec.shape[1],vec.shape[0]))
dipvec[0] = np.dot(R[:,:,0],vec.T)
dipvec[1] = np.dot(R[:,:,1],vec.T)
for i,xi in enumerate(x):
    dipvec[0,:,i] *= 1/np.sqrt(dipvec[0,0,i]**2 + dipvec[0,1,i]**2)
    dipvec[1,:,i] *= 1/np.sqrt(dipvec[1,0,i]**2 + dipvec[1,1,i]**2)

# Scale by dipole magnitude (amp in nAm)
dipvec *= magnitude


In [ ]:
### Calculate device voltage arrays
"""Voltage Projection Loop:
We iterate over devices, electrodes, sulcus surface samples (x index), and depths (z index) to compute induced voltages.
Lead Field Access:
fields[px, pz, py, :, e] -> 3-component field vector for electrode e at voxel (px, pz, py).
Dipole Vector:
Using dipvec[d,:,ix] (currently 2D rotated normals later expanded to 3D with zero y component) we project: voltage = LF · dipvec.
Performance:
Nested loops are explicit for clarity; vectorization could reduce runtime (reshape fields & broadcast). Retained due to modest grid sizes.
Array Shapes:
- dippos: [device, 2(x,y), sample]
- dipvec (after expansion below): [device, 3(x,y,z), sample]
- v: [device, sample_x, depth_z, electrode]
Scaling:
Voltages converted to microvolts (uV) assuming input dipole magnitude units.
"""
# Depth extent (mm) along z axis; resolution matches 'scale'
zstart = -8
zstop = 8
znum = int((zstop-zstart)/scale)+1
zmm = np.linspace(zstart,zstop,znum)
z = zmm/scale # scale z
z += fields.shape[0]//2 # set relative to LF corner
z = z.astype('int')

# voltage array of the shape [device,x,z,channel]; y is left out as it is defined by x
v = np.zeros((2,x.shape[0],z.shape[0],fields.shape[-1]))
# Swap axes and add a z axis to dipole vector; y -> z, new y is all 0
temp = np.zeros((2,3,x.shape[0])) # [device,3d vector,x position]
temp[:,0] += dipvec[:,0]
temp[:,2] += dipvec[:,1]
dipvec = np.copy(temp)
del temp

# Calculate voltages
for d in range(2): # d for device 0 or 1
    for e in range(fields.shape[-1]): # `e` for each electrode ID
        for ix,xi in enumerate(x):
            for iz,zi in enumerate(z):
                px = dippos[d,0,ix]
                py = dippos[d,1,ix]
                pz = zi
                LF = fields[px,pz,py,:,e]
                v[d,ix,iz,e] = np.dot(LF,dipvec[d,:,ix])

v *= 10**6 # Scale to uV units
v = np.nan_to_num(v) # fill zeros in place of NaN

## Settings for Plots


In [ ]:
# Plot & Analysis Settings
"""Parameter Definitions:
SH: If True, convert SNR maps to Shannon–Hartley information capacity (bits/s); else plot raw SNR.
montage: When True, derive SNR from peak electrode differences (max-min) per spatial point (contrast enhancement).
nrows: Number of vertical sensor rows combined into one SEEG ring (aggregation factor); affects spatial smoothing & SNR.
snr_max: Upper clipping threshold for SNR visualization (-1 disables clipping).
snr_min: Floor threshold setting sub-minimum SNR to 0 (noise indistinguishable region suppression).
sample_freq: Sampling rate (Hz) used for Shannon–Hartley bandwidth.
bw: Effective bandwidth; typically Nyquist (sample_freq/2) for capacity mapping.
color: Colormap for primary scalar fields (SNR or capacity).
colordiff: Colormap used for ratio/difference visualization (diverging for emphasis).
Adjustments:
- Increase nrows to simulate thicker SEEG contact grouping (reduces peak SNR but smooths spatial map).
- Toggle SH for physical interpretability (bits/s) vs raw detection strength.
"""
SH = True  # False -> Use SNR; True -> Use Shannon-Hartley information capacity
montage = False  # Apply montage-based difference SNR extraction if True
nrows = 2  # Rows aggregated per SEEG ring; powers of 2 recommended (2,4,8)
snr_max = -1  # Clip SNR upper bound for plotting; -1 disables clipping
snr_min = 0  # Any SNR below this threshold forced to 0 (noise floor)
sample_freq = 2000  # DiSc sampling frequency (Hz)
bw = sample_freq / 2  # Effective bandwidth for Shannon-Hartley
color = cm.rainbow  # Colormap for scalar surfaces
colordiff = cm.bwr  # Diverging colormap for ratio/difference plots


## DiSc SNR

In [ ]:
### Calculate SNR from the voltage on two devices ###
"""DiSc SNR Computation:
Input tensor v2 shape: [device, x_sample, z_depth, electrode].
Two Modes:
- montage=True: For each spatial (x,z), compute max-min across all electrodes (per device), then take the maximum difference across devices. This emulates differential referencing to enhance localized sources.
- montage=False: Use the maximum absolute electrode voltage (over devices & electrodes) as signal amplitude.
SNR Definition:
SNR = (signal_rms / noise_rms)^2 (power ratio). Here we approximate signal_rms by peak amplitude (simplification acceptable for comparative mapping).
Outputs:
- disc_SNR: 2D map [x,z] representing device-independent strongest DiSc detectability.
Notes:
- disc_noise controls scaling; adjust to empirical rms noise for realistic capacity values.
- Potential refinement: Replace peak with robust estimator (e.g., 95th percentile) to mitigate outlier influence.
"""
# Copy voltage tensor for manipulation
v2 = np.copy(v)
if montage: # Find SNR for montage
    # Find highest and lowest voltage for each spatial point
    v2max = np.max(v2,axis=3)
    v2min = np.min(v2,axis=3)
    # Largest difference regardless of device
    v2diff = np.max(v2max - v2min,axis=0)
    # Calculate SNR from difference and DiSc noise profile
    SNR = np.square(v2diff/disc_noise)
    disc_SNR = np.copy(SNR)
else: # Find SNR for non-montage
    SNR = np.square(np.max(np.max(np.abs(v2),axis=3),axis=0)/disc_noise)
    disc_SNR = np.copy(SNR)


## SEEG SNR

In [ ]:
### Convert voltages to SEEG ###
"""SEEG Aggregation & SNR:
Aggregation Logic:
128 DiSc sensors arranged in 8 columns of 16 depths. We collapse vertical groups (nrows) into SEEG ring contacts.
Index Mapping:
- column = sensor_index // 16
- depth = sensor_index - column*16
- ring index = depth // nrows
Each ring accumulates voltages from nrows * 8 sensors (all columns at that depth band). After summation, divide by count to approximate average potential.
SNR Modes mirror DiSc logic (montage vs peak amplitude).
Outputs:
- vseeg: [device, x_sample, z_depth, ring]
- seeg_SNR: 2D map [x,z]
Normalization:
Division by (nrows*8) treats grouping as uniform averaging; alternative weighting could reflect electrode geometry.
"""
# Allocate SEEG voltage tensor (rings along depth)
vseeg = np.zeros((v.shape[0], v.shape[1], v.shape[2], 128 // (8 * nrows)))
for d in range(vseeg.shape[0]):  # device
    for s in range(v.shape[-1]):  # DiSc sensor index
        column = s // 16
        depth = s - column * 16
        vseeg[d, :, :, depth // nrows] += v[d, :, :, s]

vseeg *= 1/(nrows*8) # divide by number of added sensors per ring

### Calculate SNR from the voltage on two devices ###
"""SEEG SNR Computation mirrors DiSc approach; seeg_noise defines per-contact rms baseline."""
v2 = np.copy(vseeg)
if montage: # Find SNR for montage
    # Find highest and lowest voltage for each spatial point
    v2max = np.max(v2,axis=3)
    v2min = np.min(v2,axis=3)
    # Largest difference regardless of device
    v2diff = np.max(v2max - v2min,axis=0)
    # Calculate SNR from difference and DiSc noise profile
    SNR = np.square(v2diff/seeg_noise)
    seeg_SNR = np.copy(SNR)
else:
    SNR = np.square(np.max(np.max(np.abs(v2), axis=3), axis=0) / seeg_noise)
    seeg_SNR = np.copy(SNR)

## Plot 3D Maps

In [ ]:
### Plot SNR map ###
"""3D Visualization Details:
Grid Construction:
- X, Z from meshgrid of lateral (x) and depth (zmm) axes; Y replicates sulcal profile along depth to create a surface sheet.
Normalization Strategy:
- Compute global min/max across both device modalities (DiSc, SEEG) to allow direct color-scale comparison.
Information Capacity Mode (SH=True):
- Convert SNR to capacity using C = bw * log2(SNR + 1). This bounds low-SNR regions gracefully and amplifies differentiable high-performing zones.
Shading:
- LightSource shading (soft blend) adds depth perception without altering scalar values (facecolors pre-mapped).
Colorbar:
- Custom tick labels reflect physical units (Bits/s) or SNR depending on mode; center tick aids perceptual midpoint reference.
Scalebar:
- AnchoredSizeBar provides physical scale (5 mm) derived from model dimensions (approx axis span ~15 mm).
Export:
- Figures saved as transparent PDFs for overlay in publication layouts.
"""
# Plot surface grids
X, Z = np.meshgrid(x, zmm)
Y = np.repeat(y, znum).reshape((y.shape[0], znum)).T
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw=dict(projection='3d'), figsize=(12, 6))
ls = LightSource(270, 45)

# Apply SNR floor filtering
seeg_SNR[seeg_SNR < snr_min] = 0
disc_SNR[disc_SNR < snr_min] = 0

if SH:
    sh_disc = bw*np.log2((disc_SNR).T + 1) # Information capacity; Shannon-Hartley
    sh_seeg = bw*np.log2((seeg_SNR).T + 1)

    vmin = min(sh_disc.min(), sh_seeg.min()) # Normalize datasets to the same scale 
    vmax = max(sh_disc.max(), sh_seeg.max())
    norm = Normalize(vmin=vmin, vmax=vmax)

    rgb1 = ls.shade(sh_disc,cmap=color,vert_exag=0.1,blend_mode='soft',norm=norm)
    surf1 = ax1.plot_surface(X,Z,Y,facecolors=rgb1,antialiased=True)

    rgb2 = ls.shade(sh_seeg,cmap=color,vert_exag=0.1,blend_mode='soft',norm=norm)
    surf2 = ax2.plot_surface(X,Z,Y,facecolors=rgb2,antialiased=True)

    cbar = fig.colorbar(cm.ScalarMappable(norm=norm,cmap=color),ax=[ax1,ax2],shrink=0.5,aspect=5,ticks=[0,0.5,1],orientation='vertical')
    ticks = [vmin,(vmin+vmax)/2,vmax]
    cbar.set_ticks(ticks)
    cbar.ax.set_yticklabels([str(int(vmin)), "Bits/s", str(int(vmax))])

else:
    if snr_max > 0:
        snr_plot_disc = np.where(disc_SNR > snr_max, snr_max, disc_SNR)
        snr_plot_seeg = np.where(seeg_SNR > snr_max, snr_max, seeg_SNR)
    else:
        snr_plot_disc = np.copy(disc_SNR)
        snr_plot_seeg = np.copy(seeg_SNR)

    vmin = min(snr_plot_disc.min(), snr_plot_seeg.min()) # Normalize datasets to the same scale 
    vmax = max(snr_plot_disc.max(), snr_plot_seeg.max())
    norm = Normalize(vmin=vmin, vmax=vmax)

    rgb1 = ls.shade(snr_plot_disc.T,cmap=color,vert_exag=0.1,blend_mode='soft',norm=norm)
    surf1 = ax1.plot_surface(X,Z,Y,facecolors=rgb1,antialiased=True)

    rgb2 = ls.shade(snr_plot_seeg.T,cmap=color,vert_exag=0.1,blend_mode='soft',norm=norm)
    surf2 = ax2.plot_surface(X,Z,Y,facecolors=rgb2,antialiased=True)

    cbar = fig.colorbar(cm.ScalarMappable(norm=norm,cmap=color),ax=[ax1,ax2],shrink=0.5,aspect=5,ticks=[0,0.5,1],orientation='vertical')
    ticks = [vmin,(vmin+vmax)/2,vmax]
    cbar.set_ticks(ticks)
    cbar.ax.set_yticklabels([str(int(vmin)), "Bits/s", str(int(vmax))])

# Shared figure cosmetic adjustments
fontprops = fm.FontProperties(size=10)
scalebar = AnchoredSizeBar(ax1.transData,0.038,'5 mm','lower center',
                           pad=0.1,color='black',frameon=False,
                           size_vertical=0.001,fontproperties=fontprops)
scalebar2 = AnchoredSizeBar(ax2.transData,0.038,'5 mm','lower center',
                           pad=0.1,color='black',frameon=False,
                           size_vertical=0.001,fontproperties=fontprops)
for ax in [ax1, ax2]:
    ax.set_xlim([-7.5, 7.5])
    ax.set_ylim([-7.5, 7.5])
    ax.axis('off')
    ax.grid(False)

ax1.add_artist(scalebar)
ax2.add_artist(scalebar2)
ax1.set_title("Central Sulcus Information Capacity DiSc" if SH else "Central Sulcus SNR DiSc")
ax2.set_title("Central Sulcus Information Capacity SEEG" if SH else "Central Sulcus SNR SEEG")
plt.savefig(path.join(output, '5_CSmap3d.pdf'), transparent=True)
plt.show()

## Plot 2D Heatmaps

In [ ]:
### Plot 2D heatmap of the disc data ###
plt.imshow(sh_disc, cmap = color, aspect = 1, extent = [0, 32, 0, 16],norm=norm)
plt.savefig(path.join(output,'5_CSmap2dDiSc.pdf'),transparent=True,dpi=300)
plt.show()

In [ ]:
### Plot 2D heatmap of the SEEG data ###
plt.imshow(sh_seeg, cmap = color, aspect = 1, extent = [0, 32, 0, 16],vmax=np.max(bw*np.log2((disc_SNR).T + 1)))
plt.savefig(path.join(output,'5_CSmap2dSEEG.pdf'),transparent=True,dpi=300)
plt.show()

## Difference between the two maps


In [ ]:
### Do the same but with the difference
"""Ratio Map (DiSc vs SEEG):
Definition:
- SNR Ratio: disc_SNR / seeg_SNR (or capacity ratio when SH=True via log2 transform).
Interpretation:
- Values > 1: Regions where DiSc outperforms SEEG (higher detectability or capacity).
- Values < 1: SEEG advantage zones (possibly due to spatial averaging benefits).
Capacity Mode:
- Using log2(SNR+1) stabilizes high dynamic ranges; ratio of capacities approximates relative channel throughput.
Color Scale:
- Diverging colormap centered near 1 (neutral). Tick labels annotate modality dominance extremes.
Cautions:
- Near-zero denominators inflate ratio; floor filtering before this step mitigates extreme artifacts.
- Ratio does not imply absolute superiority for clinical utility—context (peak vs integrated coverage) matters.
"""
SNR = disc_SNR / seeg_SNR

# Plot
X, Z = np.meshgrid(x,zmm)
Y = np.repeat(y,znum).reshape((y.shape[0],znum)).T
fig,ax = plt.subplots(subplot_kw=dict(projection='3d'))
ls = LightSource(270,45)
if SH:
    sh = np.log2((disc_SNR).T + 1) / np.log2((seeg_SNR).T + 1) # Information capacity; Shannon-Hartley
    vmin, vmax = 0., 2.0
    norm = Normalize(vmin=vmin, vmax=vmax)
    rgb = ls.shade(sh, cmap=colordiff, vert_exag=0.1, blend_mode='soft', norm=norm)
    surf = ax.plot_surface(X, Z, Y, facecolors=rgb, antialiased=True)
    cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=colordiff), ax=plt.gca(), shrink=0.5, aspect=5, ticks=[0.5, 1, 1.5], orientation='vertical')
    cbar.ax.set_yticklabels(['0.5 (SEEG Dom.)', 'Information Ratio (Arb.)', '1.5 (DiSc Dom.)'])
    plt.title("Information Capacity Ratio: DiSc / SEEG")
else:
    if snr_max > 0:
        snr_plot = np.where(SNR > snr_max, snr_max, SNR)
    else:
        snr_plot = np.copy(SNR)
    rgb = ls.shade(snr_plot.T,cmap=colordiff,vert_exag=0.1,blend_mode='soft')
    surf = ax.plot_surface(X,Z,Y,facecolors=rgb,antialiased=True)
    cbar = fig.colorbar(cm.ScalarMappable(cmap=colordiff),ax = plt.gca(),shrink=0.5,aspect=5,ticks=[0,0.5,1],orientation='vertical')
    cbar.ax.set_yticklabels([str(int(np.min(snr_plot))),"SNR",str(int(np.max(snr_plot)))])
    plt.title("SNR Ratio: DiSc / SEEG")

# Add scale bar and cosmetic tweaks
fontprops = fm.FontProperties(size=10)
scalebar = AnchoredSizeBar(ax.transData,0.038,'5 mm','lower center',
                           pad=0.1,color='black',frameon=False,
                           size_vertical=0.001,fontproperties=fontprops)
ax.add_artist(scalebar)
plt.xlim([-7.5, 7.5])
plt.ylim([-7.5, 7.5])
plt.axis('off')
ax.grid(False)
plt.savefig(path.join(output,'5_CSmap3dDiff.pdf'),transparent=True,dpi=300)
plt.show()

In [ ]:
### Plot 2D heatmap of the same data ###
plt.imshow(sh, cmap = colordiff, aspect = 1, extent = [0, 32, 0, 16])
plt.savefig(path.join(output,'5_CSmap2dDiff.pdf'),transparent=True,dpi=300)
plt.show()

## Printout Results

In [ ]:
# Calculate Shannon-Hartley information capacity for DiSc and SEEG
"""Final Metric Extraction:
sh_disc / sh_seeg:
- Capacity maps (bits/s) derived from SNR via Shannon–Hartley formula using bandwidth 'bw'.
max_*:
- Peak local channel throughput (not total ROI performance). Helpful for identifying focal optimal placement.
total_*:
- Sum across all samples (integrated spatial capacity). Reflects aggregate information potential over mapped region.
info_ratio:
- Pointwise relative advantage field (DiSc vs SEEG). Values >1 indicate DiSc superiority.
Indices:
- max_idx / min_idx provide spatial coordinates (x depth grid indices) for extreme ratios.
Note:
- Summation does not adjust for spatial sampling density differences; treat as relative metric only.
"""
sh_disc = bw * np.log2((disc_SNR).T + 1)
sh_seeg = bw * np.log2((seeg_SNR).T + 1)

# Maximum information capacity
max_disc = np.max(sh_disc)
max_seeg = np.max(sh_seeg)

# Total (sum) information capacity
total_disc = np.sum(sh_disc)
total_seeg = np.sum(sh_seeg)

# Ratio map
info_ratio = sh_disc / sh_seeg
max_ratio = np.max(info_ratio)
min_ratio = np.min(info_ratio)

print(f"DiSc max info capacity: {max_disc:.2f} bits/s")
print(f"SEEG max info capacity: {max_seeg:.2f} bits/s")
print(f"DiSc total info capacity: {total_disc:.2f} bits/s")
print(f"SEEG total info capacity: {total_seeg:.2f} bits/s")
# Find the indices of the max and min ratio
max_idx = np.unravel_index(np.argmax(info_ratio), info_ratio.shape)
min_idx = np.unravel_index(np.argmin(info_ratio), info_ratio.shape)

print(f"Largest DiSc/SEEG info ratio: {max_ratio:.2f} at {max_idx} (DiSc: {sh_disc[max_idx]:.2f} bits/s, SEEG: {sh_seeg[max_idx]:.2f} bits/s)")
print(f"Smallest DiSc/SEEG info ratio: {min_ratio:.2f} at {min_idx} (DiSc: {sh_disc[min_idx]:.2f} bits/s, SEEG: {sh_seeg[min_idx]:.2f} bits/s)")